## Universidade Federal do Rio Grande do Sul (UFRGS)   
Programa de Pós-Graduação em Engenharia Civil (PPGEC)   

# PEC00025: Introduction to Vibration Theory


### Class 12 - Forced vibration of multi degree of freedom systems

[1.   The Rayleigh quotient](#section_1)  
[2.   The damping matrix](#section_2)  
[3.   Modal superposition](#section_3)  
[4.   Modal analysis for static response](#section_4)  
[5.   Assignment](#section_5)  

---
_Prof. Marcelo M. Rocha, Dr.techn._ [(ORCID)](https://orcid.org/0000-0001-5640-1020)  
_Porto Alegre, RS, Brazil_ 


In [8]:
# Importing Python modules required for this notebook
# (this cell must be executed with "shift+enter" before any other Python cell)

import numpy as np
import matplotlib.pyplot as plt
import pickle as pk

# Load matrices generated in Class 10 and 11 (those notebooks must be run firstly!)

with open('resources/data/sample_KM.pk', 'rb') as KM:    # mass and stiffness matrices
    K1, M1, K2, M2, K3, M3 = pk.load(KM)

with open('resources/data/sample_VM.pk', 'rb') as VM:    # modal properties
    wk1, Phi1, wk2, Phi2, wk3, Phi3 = pk.load(VM)


## 1. The Rayleigh quotient <a name="section_1"></a> 

In a previous class we have seen that for a discrete system the potential
elastic and the kinetic energy can be given in matrix notation as:

\begin{align*}
 V &= \frac{1}{2} \; \vec{u}^{\intercal} \mathbf{K} \,\vec{u} \\
 T &= \frac{1}{2} \; \dot{\vec{u}}^{\intercal} \mathbf{M} \, \dot{\vec{u}}
\end{align*}

These expressions are valid both for each (finite) element, as for the complete system as well.
Now we recall the free vibration response in the $k$-th vibration mode and the corresponding velocity:

\begin{align*}
 \vec{u}_k(t)       &=          u_{k0} \sin \left( \omega_k t + \theta_k \right) \, \vec{\varphi}_k \\
 \vec{\dot{u}}_k(t) &= \omega_k u_{k0} \cos \left( \omega_k t + \theta_k \right) \, \vec{\varphi}_k
\end{align*}

Disregarding any energy dissipation (damping), both energy forms, $V$ and $T$, must have the same 
maximum amplitude occurring whenever the sine or the cosine function peaks at unity: 

$$ \frac{V_{\rm max}}{T_{\rm max}} =  
 \frac{     u_{k0}^2 \; \left(\vec{\varphi}_k^{\intercal} \mathbf{K} \,\vec{\varphi}_k \right)}
{\omega_k^2 u_{k0}^2 \; \left(\vec{\varphi}_k^{\intercal} \mathbf{M} \,\vec{\varphi}_k \right)} = 1 $$

This means that:

$$ \omega_k^2 = \frac{V_{\rm max}}{T_{\rm max}^*} $$

where $T_{\rm max}^*$ is called _reference kinetic energy_, which is the kinetic energy calculated
with displacements instead of velocities. 

The equation above is called _Rayleigh Quotient_, and the equality holds only for free vibration at a given 
mode $k$, whatever the vibration amplitude $u_{k0}$ is.  
However, it can be shown that _any displacement vector, $\vec{u}$, that minimizes the quotient 
above IS a vibration mode_. 
This means that by making an educated guess about $\vec{u}_k$ for mode $\vec{\varphi}_k$ and calculating 
the corresponding energies, $V_{\rm max}$ and $T_{\rm max}^*$, will provide an estimation for $\omega_k$ 
that is equal or larger the correct value.

In the following example, we use the deformed shape for a central load calculated with FTool for the
plane truss (example 1) in [Class 10](https://nbviewer.jupyter.org/github/mmaiarocha/PEC00025/blob/master/Class_10_FromSingleToMany.ipynb?flushcache=true).
For the applied load of 1.0kN, the deformed shape is (the central line of the flexibility matrix):

<img src="images/truss12central.png" alt="Central load" width="780px"/>


In [11]:
uk = 1e-3*np.array([[0.1303, 0.2325, 0.2833, 0.2325, 0.1303]]).T    # initial guess for eigenvector (m)


The maximum potential elastic energy can be calculated as the work done by external forces, while the maximum
reference kinetic energy is calculated with the (lumped) mass matrix:


In [31]:
Vmax = 1000*uk[2,0]/2                         # work done by central load 1.0kN
Tref = np.dot(uk.T, np.dot(M1, uk))[0,0]/2    # reference kinetic with mass matrix

# Eigenvalue estimation
fk = np.sqrt(Vmax/Tref)/2/np.pi
f1 = wk1[0]/(2*np.pi)

print('Frequency from Rayleigh quotient: {0:4.2f}Hz'.format(fk)) 
print('Previously calculated with scipy: {0:4.2f}Hz'.format(f1))
print('Estimation error:                 {0:4.2f}% '.format(100*(fk - f1)/f1))


Frequency from Rayleigh quotient: 4.02Hz
Previously calculated with scipy: 3.94Hz
Estimation error:                 2.02% 


This shows that the Rayleigh quotient may provide a very good estimation of natural frequencies whenever a 
convincing modal shape is assumed and the corresponding system stiffness is available.


## 2. The damping matrix <a name="section_2"></a> 

So far the natural frequencies and the corresponding modal shapes have been calculated without considering
any energy dissipation. It is however necessary to introduce a damping model, and we shall keep all the
mathematical convenience of viscous damping, that implies a reactive force proportional to velocity. 
In matrix notation the damped equilibrium equation is:

$${\mathbf M} \ddot{\vec{u}} + {\mathbf C} \dot{\vec{u}} + {\mathbf K} \vec{u} = \vec{F}(t)$$

In the next section we introduce the modal superposition method for solving linear systems subjected to
forced vibration. The method take advantage of the mode orthogonality with respect to both the mass and stiffness
matrices. To keep this advantage for damped systems, the damping matrix must also respect the orthogonality
condition, what is a quite restrictive requirement. The most practical approach for ensuring this orthogonality
is defining the damping matrix as a linear combination of mass and stiffness:

$$ {\mathbf C} = \alpha_0 {\mathbf M} + \alpha_1 {\mathbf K} $$

This means that:


$$ C_k = \alpha_0 \left( \vec{\varphi}_k^{\intercal} {\mathbf M} \vec{\varphi}_k \right)  + 
         \alpha_1 \left( \vec{\varphi}_k^{\intercal} {\mathbf K} \vec{\varphi}_k  \right) = 
         \alpha_0 M_k + \alpha_1 K_k $$

where $C_k$, $M_k$ and $K_k$ are called modal damping, modal mass and modal stiffness, respectivelly.
Now we observe that the same definitions used for a one degree of freedom system can be applied to these 
modal properties: 

$$ C_k = 2 \zeta_k M_k \omega_k = \alpha_0 M_k + \alpha_1 M_k \omega_k^2 $$

Dividing all terms by $2 M_k \omega_k$ gives the modal damping ratio:

$$ \zeta_k = \alpha_0 \left( \frac{1}{2 \omega_k} \right) + \alpha_1 \left( \frac{\omega_k}{2} \right) $$

what means the modal damping can be estimated directly from the modal frequency, such that once two
pairs $(\zeta_i, \omega_i)$ and $(\zeta_j, \omega_j)$ are specified, the complete damping matrix is defined 
from the solution of an equation system for $\alpha_0$ and $\alpha_1$.

As an example, let us take the truss above and specify a damping ratio of 1% for the second and the fourth modes.
The calculation is done as shown below:


## 3. Modal Superposition <a name="section_3"></a> 

Por se tratar de um sistema linear (ou linearizado), a equação de equilíbrio dinâmico pode ser resolvida por um método denominado superposição modal, sendo que a resposta total é obtida através da soma das respostas em cada modo de vibração. Subtituindo-se a $\vec{u}(t)$ na equação de equilíbrio e pré-multiplicando-se todos os termos por $\vec\phi_k^{\; \rm T}$ (com o $\intercal$ denotando _transposição_) tem-se:

$$
(\vec\phi_k^{\intercal}{\bf M} \vec\phi_k) \, \ddot{u}_k + 
(\vec\phi_k^{\intercal}{\bf C}  \vec\phi_k) \, \dot{u}_k + 
(\vec\phi_k^{\intercal}{\bf K}  \vec\phi_k) \,  u_k = \vec\phi_k^{\intercal} \vec{F}(t)
$$

Devido à propriedade de ortogonalidade dos autovetores $\vec \phi_k$, e desde que a matriz de amortecimento, ${\bf C}$, possa ser representada como uma combinação linear das matrizes ${\bf M}$ e ${\bf K}$ (denominada _matriz de amortecimento proporcional_, ou de Rayleigh), todos os termos da equação acima resultam escalares e pode-se escrever:

$$M_k \ddot{u}_k + C_k \dot{u}_k + K_k  u_k = F_k(t)$$

onde $M_k$, $C_k$, $K_k$ e $u_k(t)$ são denominados _massa, amortecimento, rigidez e resposta modais_, respectivamente. Deve-se dar um destaque especial à chamada _força modal_:

$$F_k(t) = \vec\phi_k^{\; \rm T} \vec{F}(t)$$

já que o método de análise dinâmica da NBR-6123 traz implícito um espectro de potência para esta função escalar do tempo, que será apresentado na próxima seção. 


## 4. Modal analysis for static response <a name="section_4"></a> 

<img src="images/simply_supported.png" alt="simply supported beam" width="400px"/>


In [ ]:
L  = 1         # bar length (m)
EI = 17.96     # bending stiffness (Nm2)
mu = 0.260     # mass per unity length (kg/m)
q  = 20000     # distributed load (N/m)

# Proposed modal shape for first mode
x  = np.linspace(0, L, 200)
qk = np.sin(np.pi*x/L)

plt.figure(1, figsize=(8,2))
plt.plot(x, qk)
    
plt.xlim( 0.0, L  );  plt.xlabel('x');
plt.ylim(-0.5, 1.5);  plt.ylabel('phi(x)');
plt.title('Proposed modal shape')
plt.grid(True)


In [ ]:
wk = np.sqrt(EI/mu)*(np.pi/L)**2
Mk = np.sum(mu*qk*qk)
Kk = wk*wk*Mk

print('Available fundamental vibration frequency: {0:7.1f} rad/s'.format(wk))
print('Modal mass (integrated over bar length):   {0:7.1f} kg'.format(Mk))
print('Modal stiffness (from mass and frequency): {0:7.0f} N/m'.format(Kk))


In [ ]:
Fk = np.sum(q*qk)         # modal force
uk = Fk/Kk                # modal displacement
up = np.max(uk*qk)        # maximum displacement

u0 = (5*q*L**4)/(384*EI)  # theoretical maximum displacement

print('Maximum displacement approximation: {0:5.2f}m'.format(up))
print('Theoretical maximum displacement:   {0:5.2f}m'.format(u0))


In [ ]:
u    = uk*qk
V    = np.sum(q*u)
Tref = np.sum(mu*u*u)
wk   = np.sqrt(V/Tref)

print('Fundamental frequency from Rayleigh quotient: {0:7.1f} rad/s'.format(wk))


## 5. Assignments <a name="section_5"></a> 

